In [8]:
# coding=utf-8
import numpy as np
import os
import pandas as pd
import csv
import sqlite3 as lite
from bs4 import BeautifulSoup
from selenium import webdriver
from shutil import copyfile
import requests
import random
import sys
modnyeslova_csv = "modnyeslova.csv"

mainLink = 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/'

In [8]:
!wget {mainLink}

--2019-04-20 18:00:13--  https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/
Resolving xn----8sbfgf1bdjhf5a1j.xn--p1ai (xn----8sbfgf1bdjhf5a1j.xn--p1ai)... 87.236.16.117
Connecting to xn----8sbfgf1bdjhf5a1j.xn--p1ai (xn----8sbfgf1bdjhf5a1j.xn--p1ai)|87.236.16.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: <<index.html>>

index.html              [  <=>               ]  55.81K   227KB/s    in 0.2s    

2019-04-20 18:00:14 (227 KB/s) - <<index.html>> saved [57145]



In [6]:
!open .

In [2]:
with open ('index.html', 'r', encoding = "ISO-8859-1") as main_html_file:
    main_html = main_html_file.read()
main_soup = BeautifulSoup(main_html)

In [5]:
theme_links = {}

main_soup.find_all('ul > li')
for ultag in main_soup.find_all('ul', {'class': 'lmenu reset'}):
    for a in ultag.find_all('a'):
        theme_links[mainLink + a['href']] = a.text.encode('ISO-8859-1').decode('WINDOWS-1251')

In [6]:
theme_links

{'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/anime': 'Аниме',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/inostrannyiy-sleng': 'Английский сленг',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/armeyskie-zhargonizmyi': 'Армейские жаргонизмы',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/abbreviaturyi': 'Аббревиатуры',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/v-zhizni': 'В жизни',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/geymeryi': 'Геймеры',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/internet': 'Интернет',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/koreyskiy-sleng': 'Корейский сленг',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/muzyika-i-video': 'Музыка и Видео',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/modniki': 'Модники',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/narkomanyi': 'Наркоманы',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/obrazovanie-i-nauka': 'Образование и наука',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/o-jzenschinah': 'O Женщинах',
 'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/o-sekse': '

In [44]:
with open ('theme_links', 'w') as links_file:
    for link, name in theme_links.items():
        links_file.write('{}\n    out={}\n'.format(link, name))

In [45]:
!open .

In [47]:
!aria2c -i theme_links -d theme_pages --continue=true


04/20 18:22:06 [NOTICE] Downloading 27 item(s)
[DL:0B][#37b2cd 0B/0B][#5de447 0B/0B][#2dfc49 0B/0B][#79c36b 0B/0B][#b61cca 0B/
04/20 18:22:08 [NOTICE] Download complete: theme_pages/Аниме

04/20 18:22:08 [NOTICE] Download complete: theme_pages/Аббревиатуры

04/20 18:22:08 [NOTICE] Download complete: theme_pages/Английский сленг
[DL:184KiB][#2dfc49 31KiB/0B][#b61cca 31KiB/0B][#02283b 0B/0B][#e2ceff 0B/0B][#
04/20 18:22:08 [NOTICE] Download complete: theme_pages/Геймеры

04/20 18:22:08 [NOTICE] Download complete: theme_pages/Армейские жаргонизмы

04/20 18:22:08 [NOTICE] Download complete: theme_pages/В жизни

04/20 18:22:08 [NOTICE] Download complete: theme_pages/Корейский сленг

04/20 18:22:09 [NOTICE] Download complete: theme_pages/Интернет

04/20 18:22:09 [NOTICE] Download complete: theme_pages/Музыка и Видео

04/20 18:22:09 [NOTICE] Download complete: theme_pages/Модники

04/20 18:22:09 [NOTICE] Download complete: theme_pages/Наркоманы

04/20 18:22:09 [NOTICE] Download complete: the

In [62]:
theme_pages_count = {}

pages_dir = '/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/modnye_slova/theme_pages/'
for page_name in os.listdir(pages_dir):
    with open(pages_dir + page_name, 'r', encoding = "ISO-8859-1") as word_page_file:
        word_page = word_page_file.read()
    soup = BeautifulSoup(word_page, "lxml")
    pages_links = soup.find('div', class_="navigation")
    last_link = pages_links.find_all('a')[-1]
    theme_pages_count[page_name] = int(last_link.text)

In [83]:
!mkdir themes
for page_name in os.listdir(pages_dir):
    os.mkdir('themes/'  + page_name)

In [7]:
theme_to_link = {v: k for k, v in theme_links.items()}

In [84]:
aria2c_input_file = ''
theme_to_link = {v: k for k, v in theme_links.items()}
for theme, link in theme_to_link.items():
    for i in range(1, theme_pages_count[theme] + 1):
        aria2c_input_file += '{}/page/{}\n    dir=themes/{}\n    out={}{}\n'.format(link, i, theme, theme, i)
        
with open ('theme_pages_links', 'w') as links_file:
    links_file.write(aria2c_input_file)

In [85]:
!aria2c -i theme_pages_links --continue=true


04/20 18:54:52 [NOTICE] Downloading 823 item(s)
[DL:0B][#ac4cbc 0B/0B][#939382 0B/0B][#172519 0B/0B][#dc6f13 0B/0B][#0eb9cf 0B/
04/20 18:54:54 [NOTICE] Download complete: themes/Аниме/Аниме1
[DL:153KiB][#939382 31KiB/0B][#172519 15KiB/0B][#dc6f13 31KiB/0B][#0eb9cf 31KiB
04/20 18:54:54 [NOTICE] Download complete: themes/Аниме/Аниме2

04/20 18:54:55 [NOTICE] Download complete: themes/Аниме/Аниме4

04/20 18:54:55 [NOTICE] Download complete: themes/Аниме/Аниме5

04/20 18:54:55 [NOTICE] Download complete: themes/Аниме/Аниме3

04/20 18:54:55 [NOTICE] Download complete: themes/Аниме/Аниме6

04/20 18:54:55 [NOTICE] Download complete: themes/Аниме/Аниме7

04/20 18:54:55 [NOTICE] Download complete: themes/Аниме/Аниме8
[DL:204KiB][#e3b3e1 31KiB/0B][#0b1a25 15KiB/0B][#636917 31KiB/0B][#d0ce3f 0B/0B
04/20 18:54:55 [NOTICE] Download complete: themes/Аниме/Аниме9

04/20 18:54:55 [NOTICE] Download complete: themes/Аниме/Аниме10

04/20 18:54:56 [NOTICE] Download complete: themes/Аниме/Аниме11

04/20 1


04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиатуры/Аббревиатуры1
[DL:378KiB][#0a721f 0B/0B][#dc5163 0B/0B][#01cc5c 0B/0B][#99d57e 0B/0B][#a3eff4
04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиатуры/Аббревиатуры3

04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиатуры/Аббревиатуры4

04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиатуры/Аббревиатуры6

04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиатуры/Аббревиатуры7

04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиатуры/Аббревиатуры9

04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиатуры/Аббревиатуры8

04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиатуры/Аббревиатуры10

04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиатуры/Аббревиатуры2

04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиатуры/Аббревиатуры5

04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиатуры/Аббревиатуры11

04/20 18:55:02 [NOTICE] Download complete: themes/Аббревиату


04/20 18:55:07 [NOTICE] Download complete: themes/В жизни/В жизни58

04/20 18:55:07 [NOTICE] Download complete: themes/В жизни/В жизни59

04/20 18:55:07 [NOTICE] Download complete: themes/В жизни/В жизни60

04/20 18:55:07 [NOTICE] Download complete: themes/В жизни/В жизни62

04/20 18:55:07 [NOTICE] Download complete: themes/В жизни/В жизни61
[DL:566KiB][#724929 31KiB/0B][#b653a4 0B/0B][#fd0891 0B/0B][#6e1a26 0B/0B][#4c8
04/20 18:55:08 [NOTICE] Download complete: themes/В жизни/В жизни63

04/20 18:55:08 [NOTICE] Download complete: themes/В жизни/В жизни64

04/20 18:55:08 [NOTICE] Download complete: themes/В жизни/В жизни65

04/20 18:55:08 [NOTICE] Download complete: themes/В жизни/В жизни66

04/20 18:55:08 [NOTICE] Download complete: themes/В жизни/В жизни67

04/20 18:55:08 [NOTICE] Download complete: themes/В жизни/В жизни69

04/20 18:55:08 [NOTICE] Download complete: themes/В жизни/В жизни68

04/20 18:55:08 [NOTICE] Download complete: themes/В жизни/В жизни70

04/20 18:55:08 [NOTICE]


04/20 18:55:13 [NOTICE] Download complete: themes/Интернет/Интернет6

04/20 18:55:13 [NOTICE] Download complete: themes/Интернет/Интернет7

04/20 18:55:13 [NOTICE] Download complete: themes/Интернет/Интернет8

04/20 18:55:13 [NOTICE] Download complete: themes/Интернет/Интернет9

04/20 18:55:13 [NOTICE] Download complete: themes/Интернет/Интернет10

04/20 18:55:14 [NOTICE] Download complete: themes/Интернет/Интернет11

04/20 18:55:14 [NOTICE] Download complete: themes/Интернет/Интернет12
[DL:617KiB][#ba42e2 0B/0B][#cfc405 0B/0B][#0f5e53 0B/0B][#12400e 0B/0B][#fb3786
04/20 18:55:14 [NOTICE] Download complete: themes/Интернет/Интернет13

04/20 18:55:14 [NOTICE] Download complete: themes/Интернет/Интернет14

04/20 18:55:14 [NOTICE] Download complete: themes/Интернет/Интернет15

04/20 18:55:14 [NOTICE] Download complete: themes/Интернет/Интернет16

04/20 18:55:14 [NOTICE] Download complete: themes/Интернет/Интернет17

04/20 18:55:14 [NOTICE] Download complete: themes/Интернет/Интернет18

0


04/20 18:55:18 [NOTICE] Download complete: themes/Музыка и Видео/Музыка и Видео5

04/20 18:55:18 [NOTICE] Download complete: themes/Музыка и Видео/Музыка и Видео7

04/20 18:55:18 [NOTICE] Download complete: themes/Музыка и Видео/Музыка и Видео6

04/20 18:55:19 [NOTICE] Download complete: themes/Музыка и Видео/Музыка и Видео8

04/20 18:55:19 [NOTICE] Download complete: themes/Музыка и Видео/Музыка и Видео9

04/20 18:55:19 [NOTICE] Download complete: themes/Музыка и Видео/Музыка и Видео10
[DL:667KiB][#195904 0B/0B][#25a25b 0B/0B][#6b83e0 0B/0B][#e03dc4 0B/0B][#8be35f
04/20 18:55:19 [NOTICE] Download complete: themes/Музыка и Видео/Музыка и Видео11

04/20 18:55:19 [NOTICE] Download complete: themes/Музыка и Видео/Музыка и Видео12

04/20 18:55:19 [NOTICE] Download complete: themes/Музыка и Видео/Музыка и Видео13

04/20 18:55:19 [NOTICE] Download complete: themes/Музыка и Видео/Музыка и Видео14

04/20 18:55:19 [NOTICE] Download complete: themes/Музыка и Видео/Музыка и Видео15

04/20 18:55:


04/20 18:55:24 [NOTICE] Download complete: themes/Образование и наука/Образование и наука15
[DL:658KiB][#5e512d 31KiB/0B][#978b89 0B/0B][#49fbaf 0B/0B][#9d319c 0B/0B][#de2
04/20 18:55:24 [NOTICE] Download complete: themes/Образование и наука/Образование и наука17

04/20 18:55:24 [NOTICE] Download complete: themes/Образование и наука/Образование и наука16

04/20 18:55:24 [NOTICE] Download complete: themes/Образование и наука/Образование и наука18

04/20 18:55:24 [NOTICE] Download complete: themes/Образование и наука/Образование и наука20

04/20 18:55:24 [NOTICE] Download complete: themes/Образование и наука/Образование и наука19

04/20 18:55:24 [NOTICE] Download complete: themes/Образование и наука/Образование и наука21

04/20 18:55:24 [NOTICE] Download complete: themes/O Женщинах/O Женщинах1

04/20 18:55:24 [NOTICE] Download complete: themes/O Женщинах/O Женщинах3

04/20 18:55:24 [NOTICE] Download complete: themes/O Женщинах/O Женщинах5

04/20 18:55:24 [NOTICE] Download complete: them


04/20 18:55:35 [NOTICE] Download complete: themes/Субкультуры/Субкультуры4

04/20 18:55:36 [NOTICE] Download complete: themes/Субкультуры/Субкультуры6

04/20 18:55:36 [NOTICE] Download complete: themes/Субкультуры/Субкультуры7

04/20 18:55:36 [NOTICE] Download complete: themes/Субкультуры/Субкультуры8

04/20 18:55:36 [NOTICE] Download complete: themes/Субкультуры/Субкультуры9

04/20 18:55:36 [NOTICE] Download complete: themes/Субкультуры/Субкультуры5

04/20 18:55:36 [NOTICE] Download complete: themes/Субкультуры/Субкультуры10

04/20 18:55:36 [NOTICE] Download complete: themes/Символы/Символы1

04/20 18:55:36 [NOTICE] Download complete: themes/Символы/Символы2
[DL:590KiB][#c0f1b8 0B/0B][#39d6f4 0B/0B][#7fd195 0B/0B][#5de9fd 0B/0B][#d1e906
04/20 18:55:36 [NOTICE] Download complete: themes/Символы/Символы3

04/20 18:55:36 [NOTICE] Download complete: themes/Символы/Символы4

04/20 18:55:36 [NOTICE] Download complete: themes/Символы/Символы7

04/20 18:55:36 [NOTICE] Download complete: them


04/20 18:55:41 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг26

04/20 18:55:41 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг28
[DL:619KiB][#8b0f6b 0B/0B][#2c0066 0B/0B][#983ef5 0B/0B][#52c2e9 0B/0B][#12b54d
04/20 18:55:41 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг29

04/20 18:55:41 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг31

04/20 18:55:41 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг33

04/20 18:55:41 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг32

04/20 18:55:41 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг30

04/20 18:55:41 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг34

04/20 18:55:41 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг35

04/20 18:55:41 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг36

04/20 18:55:41 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг37

04/20 18:55:42 [NOTICE] Downl


04/20 18:55:46 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг104

04/20 18:55:46 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг107

04/20 18:55:47 [NOTICE] Download complete: themes/Уличный Сленг/Уличный Сленг108

04/20 18:55:47 [NOTICE] Download complete: themes/Фразеологизмы/Фразеологизмы1

04/20 18:55:47 [NOTICE] Download complete: themes/Фразеологизмы/Фразеологизмы2

04/20 18:55:47 [NOTICE] Download complete: themes/Фразеологизмы/Фразеологизмы3

04/20 18:55:47 [NOTICE] Download complete: themes/Фразеологизмы/Фразеологизмы4

04/20 18:55:47 [NOTICE] Download complete: themes/Фразеологизмы/Фразеологизмы6

04/20 18:55:47 [NOTICE] Download complete: themes/Фразеологизмы/Фразеологизмы7

04/20 18:55:47 [NOTICE] Download complete: themes/Фразеологизмы/Фразеологизмы5
[DL:609KiB][#5198bc 15KiB/0B][#d9603f 0B/0B][#49afea 0B/0B][#75b563 0B/0B][#0f4
04/20 18:55:47 [NOTICE] Download complete: themes/Фразеологизмы/Фразеологизмы9

04/20 18:55:47 [NOTICE] Download c

In [11]:
soups = []
for theme in os.listdir('themes'):
    for page in os.listdir('themes/' + theme):
        with open('themes/' + theme + '/' + page, 'r', encoding = "ISO-8859-1") as words_page_file:
            soups.append((theme, BeautifulSoup(words_page_file.read())))
            

In [12]:
# <h3 class="btl"><a href="https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/411-chto-znachit-ohayo-ohae.html">Что значит Охайо, Охаё?...</a></h3>
soup = soups[0]
soup.find_all('h3', class_="btl")[0].find('a')

AttributeError: 'tuple' object has no attribute 'find_all'

In [18]:
overall = 0
for key, values in theme_words.items():
    print(key, len(values))
    overall += len(values)
    
print(overall)
print(len(words_links))

Наркоманы 80
Модники 143
Татуировки 124
О Фанфиках 36
О Сексе 90
Аниме 184
Интернет 484
Уличный Сленг 645
Тюрьма 146
Геймеры 200
Образование и наука 126
Религия 68
О Поцелуях 45
Корейский сленг 37
Числа 17
Английский сленг 217
Популярные Мемы 118
Музыка и Видео 200
О Любви 250
Символы 44
O Женщинах 150
Аббревиатуры 163
В жизни 639
Армейские жаргонизмы 39
Оскорбления 161
Субкультуры 59
Фразеологизмы 403
4868
4868


In [12]:
from collections import defaultdict

theme_words = defaultdict(list)
words_links=[]
word_link_to_themes = defaultdict(list)
for theme, soup in soups:
    for link in soup.find_all('h3', class_="btl"):
        word_link = link.find('a')['href']
        words_links.append(word_link)
        theme_words[theme].append(word_link)
        word_link_to_themes[word_link].append(theme)

In [108]:
with open ('words_pages_links', 'w') as links_file:
    links_file.write('\n'.join(words_links))

In [112]:
!aria2c -i words_pages_links -d words_pages --continue=true -x16


04/21 11:57:20 [NOTICE] Downloading 4868 item(s)
[DL:0B][#16199c 0B/0B][#7fd58a 0B/0B][#be3b99 0B/0B][#125679 0B/0B][#c8bc6a 0B/
04/21 11:57:27 [NOTICE] Download complete: words_pages/675-chto-znachit-dzhanki.html

04/21 11:57:27 [NOTICE] Download complete: words_pages/679-chto-znachit-zarolil-zarolit.html

04/21 11:57:27 [NOTICE] Download complete: words_pages/1326-fleshbek-chto-eto.html

04/21 11:57:27 [NOTICE] Download complete: words_pages/541-chto-znachit-uporotyy.html

04/21 11:57:27 [NOTICE] Download complete: words_pages/543-chto-znachit-fagat.html

04/21 11:57:27 [NOTICE] Download complete: words_pages/556-chto-znachit-flakka-flakka.html

04/21 11:57:27 [NOTICE] Download complete: words_pages/1153-kosyak-chto-znachit.html

04/21 11:57:27 [NOTICE] Download complete: words_pages/1310-splif-chto-znachit.html

04/21 11:57:27 [NOTICE] Download complete: words_pages/1038-torch-chto-znachit.html

04/21 11:57:27 [NOTICE] Download complete: words_pages/1277-lsd-chto-znachit.html
[DL:7


04/21 11:57:31 [NOTICE] Download complete: words_pages/255-chto-znachit-zamutit.html

04/21 11:57:31 [NOTICE] Download complete: words_pages/249-chto-znachit-zagnatsya-zagon.html

04/21 11:57:31 [NOTICE] Download complete: words_pages/234-chto-takoe-dyra.html

04/21 11:57:31 [NOTICE] Download complete: words_pages/228-drinchat-drink-drinch-eto.html

04/21 11:57:31 [NOTICE] Download complete: words_pages/1335-suprim-chto-znachit.html

04/21 11:57:31 [NOTICE] Download complete: words_pages/1330-snikersy-chto-znachit.html

04/21 11:57:31 [NOTICE] Download complete: words_pages/1320-merch-merche-eto.html

04/21 11:57:31 [NOTICE] Download complete: words_pages/1331-slipony-chto-znachit.html

04/21 11:57:31 [NOTICE] Download complete: words_pages/1306-tishka-chto-znachit.html

04/21 11:57:31 [NOTICE] Download complete: words_pages/1295-lofery-chto-znachit.html

04/21 11:57:31 [NOTICE] Download complete: words_pages/1633-emo-chto-znachit.html
[DL:784KiB][#be3b99 0B/0B][#8d05ce 0B/0B][#888599


04/21 11:57:35 [NOTICE] Download complete: words_pages/2977-nayki-chto-znachit.html

04/21 11:57:35 [NOTICE] Download complete: words_pages/588-chto-znachit-chelkastyy.html

04/21 11:57:35 [NOTICE] Download complete: words_pages/2959-vans-chto-znachit.html

04/21 11:57:35 [NOTICE] Download complete: words_pages/2505-zef-chto-znachit.html

04/21 11:57:35 [NOTICE] Download complete: words_pages/2937-rasta-chto-znachit.html

04/21 11:57:35 [NOTICE] Download complete: words_pages/2367-granzh-chto-znachit.html

04/21 11:57:35 [NOTICE] Download complete: words_pages/2303-alternativschiki-chto-znachit.html

04/21 11:57:35 [NOTICE] Download complete: words_pages/2236-peyting-chto-znachit.html

04/21 11:57:35 [NOTICE] Download complete: words_pages/2924-pank-chto-znachit.html

04/21 11:57:35 [NOTICE] Download complete: words_pages/2092-vanil-chto-znachit.html
[DL:807KiB][#419b1d 0B/0B][#511ba7 0B/0B][#da6742 0B/0B][#1326a3 0B/0B][#9b6b70
04/21 11:57:35 [NOTICE] Download complete: words_pages/1


04/21 11:57:38 [NOTICE] Download complete: words_pages/2782-tatu-krab-znachenie.html

04/21 11:57:39 [NOTICE] Download complete: words_pages/2785-tatu-drakona-znachenie.html

04/21 11:57:39 [NOTICE] Download complete: words_pages/2778-tatu-sobak-znachenie.html

04/21 11:57:39 [NOTICE] Download complete: words_pages/2818-tatu-zhavoronok-znachenie.html

04/21 11:57:39 [NOTICE] Download complete: words_pages/2821-tatu-yascherica-znachenie.html

04/21 11:57:39 [NOTICE] Download complete: words_pages/2819-tatu-leopard-znachenie.html

04/21 11:57:39 [NOTICE] Download complete: words_pages/2817-tatu-karp-i-koi-znachenie.html

04/21 11:57:39 [NOTICE] Download complete: words_pages/2820-tatu-lva-i-lvicy-znachenie.html

04/21 11:57:39 [NOTICE] Download complete: words_pages/2749-tatu-bykov-znachenie.html

04/21 11:57:39 [NOTICE] Download complete: words_pages/2816-tatu-nozh-znachenie.html
[DL:851KiB][#4d6cd3 0B/0B][#23013a 0B/0B][#c727e7 0B/0B][#1bf42a 0B/0B][#124f09
04/21 11:57:39 [NOTICE] Dow


04/21 11:57:44 [NOTICE] Download complete: words_pages/2815-tatu-ptic-kivi-znachenie.html

04/21 11:57:44 [NOTICE] Download complete: words_pages/2814-tatu-kenguru-znachenie.html

04/21 11:57:44 [NOTICE] Download complete: words_pages/2813-tatu-yaguar-znachenie.html

04/21 11:57:44 [NOTICE] Download complete: words_pages/2812-tatu-giena-znachenie.html

04/21 11:57:44 [NOTICE] Download complete: words_pages/2811-tatu-kolibri-znachenie.html

04/21 11:57:45 [NOTICE] Download complete: words_pages/2810-tatu-loshad-znachenie.html
[DL:764KiB][#69cffd 0B/0B][#a5bcbd 0B/0B][#c7bd47 0B/0B][#e01dec 0B/0B][#e798be
04/21 11:57:45 [NOTICE] Download complete: words_pages/2791-tatu-sokol-znachenie.html

04/21 11:57:45 [NOTICE] Download complete: words_pages/2795-tatu-ryby-znachenie.html

04/21 11:57:45 [NOTICE] Download complete: words_pages/2792-tatu-pero-znachenie.html

04/21 11:57:45 [NOTICE] Download complete: words_pages/2793-tatu-koshachih-znachenie.html

04/21 11:57:45 [NOTICE] Download compl

...


04/21 12:01:12 [NOTICE] Download complete: words_pages/1263-abiens-abi-perevod-s-latinskogo.html

04/21 12:01:12 [NOTICE] Download complete: words_pages/1262-shilo-v-meshke-ne-utaish-smysl-poslovicy.html

04/21 12:01:12 [NOTICE] File already exists. Renamed to words_pages/1236-ot-sumy-i-ot-tyurmy-ne-zarekaysya-smysl-frazeologizma.1.html.

04/21 12:01:12 [NOTICE] Download complete: words_pages/1236-ot-sumy-i-ot-tyurmy-ne-zarekaysya-smysl-frazeologizma.1.html

04/21 12:01:12 [NOTICE] Download complete: words_pages/1261-skazka-pro-belogo-bychka-znachenie-frazeologizma.html

04/21 12:01:12 [NOTICE] Download complete: words_pages/1235-pletyu-obuha-ne-pereshibesh-znachenie-poslovicy.html

04/21 12:01:12 [NOTICE] Download complete: words_pages/1233-kakov-pop-takov-i-prihod-znachenie-poslovicy.html

04/21 12:01:12 [NOTICE] Download complete: words_pages/1232-o-mertvyh-libo-horosho-libo-nichego-kto-skazal.html

04/21 12:01:12 [NOTICE] Download complete: words_pages/1234-tyazhela-ty-shapka-mono


04/21 12:01:15 [NOTICE] Download complete: words_pages/2170-beleny-obelsya-znachenie-frazeologizma.html

04/21 12:01:15 [NOTICE] Download complete: words_pages/2151-nevziraya-na-lica-znachenie-frazeologizma.html

04/21 12:01:15 [NOTICE] Download complete: words_pages/2171-avgievy-konyushni-znachenie-frazeologizma.html

04/21 12:01:15 [NOTICE] Download complete: words_pages/2150-doyti-do-ruchki-chto-znachit.html

04/21 12:01:15 [NOTICE] Download complete: words_pages/2148-nakrylsya-mednym-tazom-otkuda-vyrazhenie.html

04/21 12:01:15 [NOTICE] Download complete: words_pages/2734-pet-difiramby-znachenie.html

04/21 12:01:15 [NOTICE] Download complete: words_pages/2735-anika-voin-znachenie.html

04/21 12:01:15 [NOTICE] Download complete: words_pages/2147-kak-korova-yazykom-slizala-znachenie-frazeologizma.html

04/21 12:01:15 [NOTICE] Download complete: words_pages/2733-myagko-stelesh-da-zhestko-spat-znachenie.html

04/21 12:01:15 [NOTICE] Download complete: words_pages/2712-nischie-duhom-z


04/21 12:01:18 [NOTICE] Download complete: words_pages/1190-pirrova-pobeda-znachenie-frazeologizma.html

04/21 12:01:18 [NOTICE] Download complete: words_pages/1191-luchshe-sinica-v-rukah-chem-zhuravl-v-nebe-znachenie.html

04/21 12:01:18 [NOTICE] Download complete: words_pages/1189-potemkinskie-derevni-znachenie-frazeologizma.html

04/21 12:01:18 [NOTICE] Download complete: words_pages/1174-znakomye-vse-lica-znachenie.html

04/21 12:01:18 [NOTICE] Download complete: words_pages/1188-enfant-terrible-perevod-vyrazheniya.html

04/21 12:01:18 [NOTICE] Download complete: words_pages/1175-kapli-datskogo-korolya-chto-takoe.html

04/21 12:01:18 [NOTICE] Download complete: words_pages/1173-darenomu-konyu-v-zuby-ne-smotryat-znachenie.html

04/21 12:01:18 [NOTICE] Download complete: words_pages/1172-unter-oficerskaya-vdova-sama-sebya-vysekla-znachenie.html

04/21 12:01:18 [NOTICE] File already exists. Renamed to words_pages/1170-dobryy-samarityanin-chto-znachit.1.html.

04/21 12:01:18 [NOTICE] 


04/21 12:01:21 [NOTICE] Download complete: words_pages/2436-ne-pominay-vsue-chto-znachit.1.html

04/21 12:01:21 [NOTICE] Download complete: words_pages/2432-pochit-v-boze-chto-znachit.html

04/21 12:01:21 [NOTICE] Download complete: words_pages/2433-kak-zenicu-oka-chto-znachit.html

04/21 12:01:21 [NOTICE] Download complete: words_pages/2440-sobaka-laet-karavan-idet-chto-znachit.html

04/21 12:01:21 [NOTICE] Download complete: words_pages/2429-oko-za-oko-chto-znachit.html

04/21 12:01:21 [NOTICE] File already exists. Renamed to words_pages/2419-vecher-v-hatu-chto-znachit.1.html.

04/21 12:01:21 [NOTICE] Download complete: words_pages/2419-vecher-v-hatu-chto-znachit.1.html

04/21 12:01:21 [NOTICE] Download complete: words_pages/2470-ni-puha-ni-pera-chto-znachit.html

04/21 12:01:21 [NOTICE] Download complete: words_pages/2463-prohodit-mednye-truby-chto-znachit.html

04/21 12:01:21 [NOTICE] Download complete: words_pages/2458-o-vremena-o-nravy-kto-skazal.html

04/21 12:01:21 [NOTICE] Do


04/21 12:01:24 [NOTICE] File already exists. Renamed to words_pages/1834-pustit-sheptuna-chto-znachit.1.html.

04/21 12:01:24 [NOTICE] Download complete: words_pages/1834-pustit-sheptuna-chto-znachit.1.html

04/21 12:01:24 [NOTICE] Download complete: words_pages/2641-kak-syr-v-masle-katatsya-znachenie.html

04/21 12:01:24 [NOTICE] Download complete: words_pages/2640-kamo-gryadeshi-chto-oznachaet.html

04/21 12:01:24 [NOTICE] Download complete: words_pages/2643-kozel-otpuscheniya-znachenie.html

04/21 12:01:24 [NOTICE] Download complete: words_pages/2639-kak-s-gusya-voda-znachenie.html

04/21 12:01:24 [NOTICE] Download complete: words_pages/2638-igra-ne-stoit-svech-znachenie.html

04/21 12:01:24 [NOTICE] Download complete: words_pages/2637-ele-ele-dusha-v-tele-znachenie.html

04/21 12:01:24 [NOTICE] Download complete: words_pages/2701-voyti-v-razh-znachenie.html

04/21 12:01:24 [NOTICE] Download complete: words_pages/2681-vysasyvat-iz-palca-znachenie.html
[DL:0.9MiB][#0340a9 0B/0B][#dc

In [ ]:
[os.remove('words_pages/' + page) for page in os.listdir('words_pages') if page.endswith('.1.html') or page.endswith('.2.html') or page.endswith('.3.html')]

In [109]:
!open .

In [122]:
len(os.listdir('words_pages'))

3935

In [86]:
pages_dir

'/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/modnye_slova/theme_pages/'

In [2]:
pages_dir = !pwd
pages_dir = pages_dir[0] + '/words_pages/'

In [142]:
theme_words['Наркоманы'][0]

'https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/679-chto-znachit-zarolil-zarolit.html'

In [ ]:
words = pd.read_csv('/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/modnyeslova2.csv')

In [227]:
def ru_text(text):
    return text.encode('cp1252').decode('WINDOWS-1251')

import re

def up_info(soup, text):
    info = soup.find('div', class_="info")
    if info:
        info = ru_text(info.text)
        text = info + '\n' + text.replace(info, '')
    return text

stop_list = ['модные-слова.рф', 
             'Прочтя эту статью, вы узнали', 
             'adaptiv_block_1',
             'Прочтите ещё несколько статьей по расшифровке', 
             'Поэтому, обязательно добавьте данный ресурс к себе в закладки',
             'Добавьте наш интересный, а местами толковый сайт',
             'Например, что значит ',
             'Однако, прежде чем', 
             'Однако, перед тем, как я продолжу',
             'Впрочем, прежде чем продолжить',
             'Впрочем, перед тем, как',
             'перед тем, как продолжить,',
            'Наш сайт предоставляет вам возможность узнать']

def clean_sentences(text):
    from nltk import sent_tokenize
    paragraphs = text.split('\n')
    for i in range(len(paragraphs)):
        para = paragraphs[i]
        sentences = sent_tokenize(para)
        clean_sentences = []
        for sentence in sentences:
            if not all([sentence.find(stop_word) == -1 for stop_word in stop_list]):
                break
            clean_sentences.append(sentence)
        paragraphs[i] = ' '.join(clean_sentences)
    return '\n'.join(paragraphs).strip()
#     paragraphs = text.split('\n')
#     for i in range(len(paragraphs)):
#         paragraphs[i] = paragraphs[i].strip()
#         for stop_paragraph in ['Впрочем, прежде чем', '@media(', '(adsbygoogle']:
#             a = paragraphs[i].find(stop_paragraph)
#             if a != -1:
#                 paragraphs[i] = paragraphs[i][:a]
        
#         sentences = paragraphs[i].split('. ')
#         sentences = [s.strip()
#                  for s in sentences 
#                  if s.strip() != '' and all([s.find(stop_word) == -1 for stop_word in stop_list])]
#         paragraphs[i] = '. '.join(sentences)
        
#     return '\n'.join(paragraphs)

google_ads = ['adaptiv_block_1 { width: 300px; height: 250px; }\n@media(min-width: 500px) { .adaptiv_block_1 { width: 468px; height: 60px; } }\n@media(min-width: 800px) { .adaptiv_block_1 { width: 580px; height: 400px; } }\n\n\n\n\n\n(adsbygoogle = window.adsbygoogle || []).push({});',
             'adsbygoogle = window.adsbygoogle || []).push({});']
def clean_google_ad(text):
    for ad in google_ads:
        text = text.replace(ad, '')
    return text

endings_stop_list = [' - что значит?', ' - перевод и значение?', ' - расшифровка и перевод', ' - что значить?', ' - перевод слова?', ' - что значит мем?', ' - это значит?', ' - кто сказал?', ' - по татарски?', ' - перевод сокращения', ' - смысл выражения?', ' - это женщина?', ' - кто такой?', ' - значение термина?', ' -  смысл пословицы?', ' - это химия или чувство?', ' - значение?', ' - значение цветка', ' - расшифровка', ' - значение слова', ' - фразеологизм значение?', ' - перевод с украинского', ' - расшифровка?', ' - хорошо или плохо?', ' - как избавиться?', ' - как вернуть?', ' - перевод, что значит?', ' - значение фразеологизма?', ' - рассуждения?', ' - что лучше?', ' - откуда выражение?', ' - перевод с латыни?', ' - значение пословицы?', ' - значение поговорки?', ' - это дар или химическая реакция?', ' - расшифровка аббревиатуры?', ' - что это значит?', ' - смысл?', ' - плюсы и минусы', ' - толкование?', ' - по корейски?', ' - перевод термина?', ' - славянская богиня', ' - что такое?', ' - стоит ли?', ' - на всю жизнь?', ' - перевод и значение', ' - где находятся?', ' - смысл поговорки', ' -  что значит?', ' - перевод выражения?', ' - как себя с ним вести?', ' - откуда фраза?', ' - кто такие?', ' - что значит', ' - кто автор?', ' - смысл пословицы?', ' - автор?', ' - расшифровка и перевод?', ' - что это?', ' - значение фразеологизма', ' - смысл пословицы', ' - перевод с арабского?', ' - значение слова?', '- перевод', ' - что делать?', ' - какие выбрать?', ' - Что значит?', ' - это какой?', ' это?', ' - значит?', ' - перевод с латыни', ' - кто такой в Clash of Clans?', ' - значение фразы?', ' - кто такая?', ' - это?', ' - перевод с латинского?', ' - кратко', ' - что означает?', ' - значение', ' - есть ли смысл?', ' - перевод?', ' - что значит слово?', ' - расшифровка аббревиатуры', ' - смысл фразеологизма?', ' - значение понятия?']
beginnings_stop_list = ['Что значит ', 'Что такое ', 'Что означает ', 'Кто такой ', 'Кто такие ']
def clean_name(name):
    if name.find('- ') != -1:
        for word in endings_stop_list:
             if name.endswith(word):
                name = name.replace(word, '')
                break
    if name.endswith(' это?'):
        name = name.replace(' это?', '')
    for word in beginnings_stop_list:
        if name.startswith(word):
            name = name.replace(word, '')
    return name.strip(' \n?')
    
def clean_highslide_caption(soup, text):   
    a = soup.find('span', class_="highslide-caption")
    if a:
        to_rm = '0\n' + ru_text(a.text)
        text = text.replace(to_rm, '')
    return text
    

def parse_word_page(page, soup):
    try:
        if not page.startswith(mainLink):
            page = mainLink + page
        word = dict()
        word['themes'] = ', '.join(word_link_to_themes[page])
        word['link'] = page[:page.rfind('/')].replace('https://', '').encode('utf-8').decode('idna') + page[page.rfind('/'):]
        word['name'] = ru_text(soup.find('h1', class_="btl").text)
        word['name'] = clean_name(word['name'])
        content = soup.find('div', class_="maincont")
        word['definition'] = ru_text(content.text)
        word['definition'] = clean_highslide_caption(soup, word['definition'])    
        word['definition'] = clean_google_ad(word['definition'])    
        word['definition'] = up_info(soup, word['definition'])
        word['definition'] = clean_sentences(word['definition'])
        word['definition'] = word['definition'].replace('\n(\n', ' ').replace('\nп\n', ' ')
        word['definition'] = word['definition'].replace(u'\xa0', u' ')
        word['definition'] = re.sub(r'\n+', '\n', word['definition'])
        word['definition'] = re.sub(r' +', ' ', word['definition'])
        word['definition'] = re.sub(r'\n +', '\n', word['definition'])
        word['definition'] = re.sub(r'\t+', ' ', word['definition'])
        if content.find('iframe'):
            word['video'] = content.find('iframe')['src']
        return word
    except UnicodeEncodeError as e:
        print(page, e)

In [231]:
# %%time

rand = random.randint(1, 3000)
words_dict = []
soups = []
UnicodeEncodeError_words = []
for i, page in enumerate(os.listdir(pages_dir)[:]):
    with open(pages_dir + page, 'r', encoding = "cp1252") as word_page_file:
        word_page = word_page_file.read()
    soup = BeautifulSoup(word_page, "lxml")
    content = soup.find('div', class_="maincont")
    for span in content.find_all(['td', 'style', 'script', 'table']):
        span.decompose()
    new_tag = soup.new_tag('h1', class_="btl", text=soup.find('h1', class_="btl").text)
    content.insert(0, new_tag)
    soups.append(content)
# for soup in soups:
#     soup = soups[i]
    word = parse_word_page(page, soup)
    if word:
        words_dict.append(word)
    else:
        UnicodeEncodeError_words.append(page)


https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2453-oksyumoron-chto-znachit.html 'charmap' codec can't encode characters in position 980-982: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/45-chto-znachit-aydol.html 'charmap' codec can't encode characters in position 3453-3454: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2292-otome-chto-znachit.html 'charmap' codec can't encode character '\u4e59' in position 1439: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3120-zri-v-koren-chto-znachit.html 'charmap' codec can't encode character '\u0463' in position 1254: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3020-imoto-chto-znachit.html 'charmap' codec can't encode character '\u3044' in position 1244: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3302-vuyki-chto-znachit.html 'charmap' codec can't encode character '\u0301' in position 2198: character maps to <undefined>
https://x

https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2420-shifrueshsya-chto-znachit.html 'charmap' codec can't encode characters in position 1045-1049: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3150-dialektika-chto-znachit.html 'charmap' codec can't encode characters in position 1026-1035: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2506-orgazm-chto-znachit.html 'charmap' codec can't encode characters in position 858-862: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3350-smeknuli-chto-znachit.html 'charmap' codec can't encode characters in position 1176-1177: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/879-panoptikum-chto-znachit.html 'charmap' codec can't encode characters in position 704-706: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3095-dodzikko-chto-znachit.html 'charmap' codec can't encode character '\u30c9' in position 1060: character maps to <undefined>
https:

https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3050-kospley-chto-znachit.html 'charmap' codec can't encode character '\u30b3' in position 1366: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2544-na-zare-chto-znachit.html 'charmap' codec can't encode characters in position 987-991: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3087-omake-chto-znachit.html 'charmap' codec can't encode character '\u304a' in position 29: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3341-bukaki-chto-znachit.html 'charmap' codec can't encode characters in position 1269-1273: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2422-nuzhda-chto-znachit.html 'charmap' codec can't encode character '\u0479' in position 918: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3083-kodomomuke-chto-znachit.html 'charmap' codec can't encode character '\u5b50' in position 1005: character maps to <undefined>
https://xn

https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2835-tatu-pixie-elf-znachenie.html 'charmap' codec can't encode characters in position 3511-3512: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2875-tatu-gepard-znachenie.html 'charmap' codec can't encode character '\u0101' in position 1826: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3014-bakunyu-chto-znachit.html 'charmap' codec can't encode characters in position 928-929: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3309-sosatsya-chto-znachit.html 'charmap' codec can't encode character '\u0301' in position 1418: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3082-kigurumi-chto-znachit.html 'charmap' codec can't encode character '\u7740' in position 1105: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3355-kayfarik-chto-znachit.html 'charmap' codec can't encode characters in position 1182-1184: character maps to <undefined>
h

https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3071-meganekko-chto-znachit.html 'charmap' codec can't encode characters in position 808-811: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2728-keltskie-simvoly-znachenie.html 'charmap' codec can't encode characters in position 7452-7453: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3080-bara-chto-znachit.html 'charmap' codec can't encode characters in position 111-112: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3624-asmr-chto-znachit.html 'charmap' codec can't encode character '\u0301' in position 1301: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2924-pank-chto-znachit.html 'charmap' codec can't encode characters in position 8207-8208: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2411-otorvatsya-chto-znachit.html 'charmap' codec can't encode character '\u0101' in position 867: character maps to <undefined>
https://xn---

https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3041-tokusacu-chto-znachit.html 'charmap' codec can't encode character '\u7279' in position 969: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2478-disciplina-hromaet-chto-znachit.html 'charmap' codec can't encode character '\u012b' in position 1033: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2441-sobaka-na-sene-chto-znachit.html 'charmap' codec can't encode characters in position 1089-1092: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2816-tatu-nozh-znachenie.html 'charmap' codec can't encode characters in position 7172-7173: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3048-pofigizm-chto-znachit.html 'charmap' codec can't encode character '\u012b' in position 1067: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2550-navazhdenie-znachenie-slova.html 'charmap' codec can't encode character '\u0465' in position 833: character 

In [232]:
print(len(UnicodeEncodeError_words))
words = pd.DataFrame(words_dict)
words.drop_duplicates(inplace=True, keep='first')

284


In [233]:
list(words.definition)[:2]

['Звезда Руси - представляет собой связь мужского и женского начала, что символизирует продолжение жизни на Земле. Идеальная структура линий передаёт победу разума над тьмой, гармонии над хаосом\nЛюди во все времена верили, что их окружают добрые и злые духи, которые пытаются воздействовать на их судьбу. Одновременно с этим, шло создание защиты от негативной энергетики, сглаза и злой ворожбы. В итоге были созданы различные символы, амулеты и обереги, которые по мнению знающих граждан способны оказать благотворное воздействие на любого индивидуума. Сегодня мы коснёмся такого интересного символа, как Звезда Руси, значение оберега вы узнаете немного ниже.\nИтак, продолжим, Звезда Руси значение оберега?\nчто значит Звезда Руси?Звезда Руси - это символ семейного очага, с 4-мя языками пламени исходящими из него (первый - это стремление к достижению цели; второй - жажда свободы; третий - помогает в защите веры и страны; четвёртый оказывает поддержку при серьёзных проблемах)\nЗвезда Руси - явл

In [234]:
words.to_csv('/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/modnyeslova3.csv', index=False, )

In [235]:
len(words)

3651

_____

In [225]:
error_words_dict = []
error_soups = []
error_UnicodeEncodeError_words = []
for page in UnicodeEncodeError_words:
    with open(pages_dir + page, 'r', errors='ignore') as word_page_file:
        word_page = word_page_file.read()
    soup = BeautifulSoup(word_page, "lxml")
    content = soup.find('div', class_="maincont")
    for span in content.find_all(['td', 'style', 'script', 'table']):
        span.decompose()
    new_tag = soup.new_tag('h1', class_="btl", text=soup.find('h1', class_="btl").text)
    content.insert(0, new_tag)
    error_soups.append(content)
# for soup in soups:
    word = parse_word_page(page, soup)
    if word:
        error_words_dict.append(word)
    else:
        error_UnicodeEncodeError_words.append(page)
        
len(error_UnicodeEncodeError_words)/len(UnicodeEncodeError_words)

https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2453-oksyumoron-chto-znachit.html 'charmap' codec can't encode characters in position 189-191: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/45-chto-znachit-aydol.html 'charmap' codec can't encode character '\u0578' in position 153: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2292-otome-chto-znachit.html 'charmap' codec can't encode character '\u4e59' in position 255: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3120-zri-v-koren-chto-znachit.html 'charmap' codec can't encode character '\u0463' in position 222: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3020-imoto-chto-znachit.html 'charmap' codec can't encode character '\u3044' in position 233: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3302-vuyki-chto-znachit.html 'charmap' codec can't encode character '\u0301' in position 401: character maps to <undefined>
https://xn-

https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3095-dodzikko-chto-znachit.html 'charmap' codec can't encode character '\u30c9' in position 197: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2946-kanut-v-letu-znachenie.html 'charmap' codec can't encode characters in position 210-213: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2992-ya-znayu-chto-nichego-ne-znayu-znachit.html 'charmap' codec can't encode characters in position 221-222: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3464-valyat-duraka-chto-znachit.html 'charmap' codec can't encode character '\u0301' in position 880: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3489-mozgoeb-chto-znachit.html 'charmap' codec can't encode character '\u011b' in position 279: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2444-yaschik-pandory-chto-znachit.html 'charmap' codec can't encode characters in position 162-168: character m

https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3352-otshelnik-chto-znachit.html 'charmap' codec can't encode character '\u0113' in position 250: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2726-harizmatichnyy-chto-znachit.html 'charmap' codec can't encode characters in position 153-159: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3620-v-topku-chto-znachit.html 'charmap' codec can't encode character '\u0101' in position 314: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2457-koshernaya-eda-chto-znachit.html 'charmap' codec can't encode characters in position 192-194: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3211-obratnyy-garem-chto-znachit.html 'charmap' codec can't encode character '\u30cf' in position 209: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3097-doraemon-chto-znachit.html 'charmap' codec can't encode characters in position 195-197: character maps to <und

https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2847-tatu-akuly-znachenie.html 'charmap' codec can't encode character '\u05f8' in position 545: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3497-skolko-let-skolko-zim-chto-znachit.html 'charmap' codec can't encode character '\u011b' in position 209: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2681-vysasyvat-iz-palca-znachenie.html 'charmap' codec can't encode characters in position 326-329: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3574-cannabis-perevod.html 'charmap' codec can't encode characters in position 274-281: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3345-omorasi-chto-znachit.html 'charmap' codec can't encode character '\u304a' in position 211: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3045-slesh-yaoy-chto-znachit.html 'charmap' codec can't encode characters in position 378-379: character maps to <undefi

https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3683-lovelas-chto-znachit.html 'charmap' codec can't encode character '\u0301' in position 493: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3068-genki-chto-znachit.html 'charmap' codec can't encode character '\u5143' in position 732: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3023-opening-chto-znachit.html 'charmap' codec can't encode characters in position 191-196: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3002-otymet-chto-znachit.html 'charmap' codec can't encode character '\u04b8' in position 22: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3037-otaku-chto-znachit.html 'charmap' codec can't encode character '\u304a' in position 254: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3295-alternativnaya-vselennaya-chto-znachit.html 'charmap' codec can't encode characters in position 1363-1364: character maps to <undefined

https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3066-idilliya-chto-znachit.html 'charmap' codec can't encode characters in position 175-183: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3482-nyaha-chto-znachit.html 'charmap' codec can't encode characters in position 339-341: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3513-feniks-znachenie.html 'charmap' codec can't encode character '\u017b' in position 547: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2345-paranoik-chto-znachit.html 'charmap' codec can't encode characters in position 172-179: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/2629-zhrebiy-broshen-znachenie.html 'charmap' codec can't encode characters in position 308-316: character maps to <undefined>
https://xn----8sbfgf1bdjhf5a1j.xn--p1ai/3488-ironiya-chto-znachit.html 'charmap' codec can't encode character '\u0338' in position 207: character maps to <undefined>
https://xn----8sbfg

1.0

In [ ]:
with open('/Users/lotfull/Desktop/RuSlangApp/SlangDictionary/modnyeslova3.csv', 'a') as f:
    df.to_csv(f, index=False, header=False)